In [1]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten
from keras.layers import Conv2D
from keras.layers import MaxPooling2D


input_shape = (224, 224, 3)
#Instantiate an empty model
model = Sequential([
Conv2D(64, (3, 3), input_shape=input_shape, padding="same", activation="relu"),
Conv2D(64, (3, 3), activation="relu", padding="same"),
MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),
Conv2D(128, (3, 3), activation="relu", padding="same"),
Conv2D(128, (3, 3), activation="relu", padding="same",),
MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),
Conv2D(256, (3, 3), activation="relu", padding="same",),
Conv2D(256, (3, 3), activation="relu", padding="same",),
Conv2D(256, (3, 3), activation="relu", padding="same",),
MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),
Conv2D(512, (3, 3), activation="relu", padding="same",),
Conv2D(512, (3, 3), activation="relu", padding="same",),
Conv2D(512, (3, 3), activation="relu", padding="same",),
MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),
Conv2D(512, (3, 3), activation="relu", padding="same",),
Conv2D(512, (3, 3), activation="relu", padding="same",),
Conv2D(512, (3, 3), activation="relu", padding="same",),
MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),
Flatten(),
Dense(4096, activation="relu"),
Dense(4096, activation="relu"),
Dense(1000, activation="softmax")
])
model.summary()
# Compile the model
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer="adam", metrics=["accuracy"])

Using TensorFlow backend.


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 224, 224, 64)      1792      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 224, 224, 64)      36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 112, 112, 64)      0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 112, 112, 128)     73856     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 112, 112, 128)     147584    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 56, 56, 128)       0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 56, 56, 256)      

In [0]:
from keras.models import Model
from keras.layers import Dense,Flatten
from keras.applications import vgg16
from keras import backend as K
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input, decode_predictions
import numpy as np 

In [3]:
model = vgg16.VGG16(weights='imagenet', include_top=True)

553467904/553467096 [==============================] - 6s 0us/step


In [0]:
model2 = Model(model.input, model.layers[-2].output)

In [10]:
!wget https://github.com/mlittmancs/great_courses_ml/raw/master/imgs/all01.jpg
!wget https://github.com/mlittmancs/great_courses_ml/raw/master/imgs/all02.jpg
!wget https://github.com/mlittmancs/great_courses_ml/raw/master/imgs/all03.jpg
!wget https://github.com/mlittmancs/great_courses_ml/raw/master/imgs/all04.jpg
!wget https://github.com/mlittmancs/great_courses_ml/raw/master/imgs/all05.jpg
!wget https://github.com/mlittmancs/great_courses_ml/raw/master/imgs/all06.jpg
!wget https://github.com/mlittmancs/great_courses_ml/raw/master/imgs/all07.jpg
!wget https://github.com/mlittmancs/great_courses_ml/raw/master/imgs/all08.jpg
!wget https://github.com/mlittmancs/great_courses_ml/raw/master/imgs/all09.jpg
!wget https://github.com/mlittmancs/great_courses_ml/raw/master/imgs/all10.jpg

--2020-05-13 11:48:55--  https://github.com/mlittmancs/great_courses_ml/raw/master/imgs/all01.jpg
Resolving github.com (github.com)... 140.82.113.3
Connecting to github.com (github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/mlittmancs/great_courses_ml/master/imgs/all01.jpg [following]
--2020-05-13 11:48:55--  https://raw.githubusercontent.com/mlittmancs/great_courses_ml/master/imgs/all01.jpg
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5629792 (5.4M) [application/octet-stream]
Saving to: ‘all01.jpg.1’

all01.jpg.1         100%[===================>]   5.37M  14.8MB/s    in 0.4s    

2020-05-13 11:48:56 (14.8 MB/s) - ‘all01.jpg.1’ saved [5629792/5629792]

--2020-05-13 11:48:5

In [11]:
!wget https://github.com/mlittmancs/great_courses_ml/raw/master/imgs/tyr01.jpg
!wget https://github.com/mlittmancs/great_courses_ml/raw/master/imgs/tyr02.jpg
!wget https://github.com/mlittmancs/great_courses_ml/raw/master/imgs/tyr03.jpg
!wget https://github.com/mlittmancs/great_courses_ml/raw/master/imgs/tyr04.jpg
!wget https://github.com/mlittmancs/great_courses_ml/raw/master/imgs/tyr05.jpg
!wget https://github.com/mlittmancs/great_courses_ml/raw/master/imgs/tyr06.jpg
!wget https://github.com/mlittmancs/great_courses_ml/raw/master/imgs/tyr07.jpg
!wget https://github.com/mlittmancs/great_courses_ml/raw/master/imgs/tyr08.jpg
!wget https://github.com/mlittmancs/great_courses_ml/raw/master/imgs/tyr09.jpg
!wget https://github.com/mlittmancs/great_courses_ml/raw/master/imgs/tyr10.jpg
!wget https://github.com/mlittmancs/great_courses_ml/raw/master/imgs/tyr11.jpg
!wget https://github.com/mlittmancs/great_courses_ml/raw/master/imgs/tyr12.jpg

--2020-05-13 11:49:58--  https://github.com/mlittmancs/great_courses_ml/raw/master/imgs/tyr01.jpg
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/mlittmancs/great_courses_ml/master/imgs/tyr01.jpg [following]
--2020-05-13 11:49:59--  https://raw.githubusercontent.com/mlittmancs/great_courses_ml/master/imgs/tyr01.jpg
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5178767 (4.9M) [image/jpeg]
Saving to: ‘tyr01.jpg’

tyr01.jpg           100%[===================>]   4.94M  14.2MB/s    in 0.3s    

2020-05-13 11:49:59 (14.2 MB/s) - ‘tyr01.jpg’ saved [5178767/5178767]

--2020-05-13 11:50:00--  https://githu

In [0]:
# get images
from PIL import Image
from IPython.display import display
         
dat = []
labs = []
imgs = []
for dino in ("all","tyr"):
  for i in range(1,11):
    dig = str(i)
    if len(dig) == 1: dig = "0"+dig
    img = image.load_img(dino+dig+".jpg", target_size=(224,224))
    imgs.append(img)
    img_arr = np.expand_dims(image.img_to_array(img), axis=0)
    x = preprocess_input(img_arr)
    preds = model2.predict(x)
    dat.append(preds[0])
    labs.append(dino == 'tyr')

In [0]:
train = [i for i in range(0,5)]+[i for i in range(10,15)]
test = [i for i in range(5,10)]+[i for i in range(15,20)]

In [17]:
for i in test:
  bestj = -1
  bestdist = 0
  for j in train:
    if i != j: 
      dist = dat[i] @ dat[j]
      if dist > bestdist: bestj, bestdist = j, dist
  print("pair {}".format(i))
  display(imgs[i])
  display(imgs[bestj])
  print(labs[i], labs[bestj])
  print(" ")

Output hidden; open in https://colab.research.google.com to view.